## Easy Experiment .ipynb File

### <font color='red'><b> 🐬 기억합시다 :] 실험마다 인덱스 붙여서 파일 새로만들기~! <b></font>

### unseen data로 validation을 측정하는 실험

## import

In [ ]:
import torch
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torch.nn.functional as F

import random
import time
from datetime import timedelta
import shutil

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_model_summary import summary

from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import multiprocessing
import timm

from PIL import Image

import os
import json

import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

## configuration

In [ ]:
### cofiguration
seed = 42
check_point_dir_name = '4hd_1'
data_dir = './dataset/train/images/'
train_csv_path = './dataset/train/custom_train.csv'
save_dir = f'./checkpoints/{check_point_dir_name}'

train_b_size = 16
valid_b_size = 300
train_ratio = 0.8
epochs = 25
print_interval = 800
lr = 1.5e-4
model_name = 'resnet34'
num_labels = 3+2+3+1
loss_function_name = 'FocalLoss' # ex FocalLoss, CrossEntropyLoss, MSELoss, HuberLoss L1LossFlat
optimizer_name = 'AdamW'
weight_decay = 0.02
scheduler_name = 'CosineAnnealingWarmRestarts'
comment = f'3head but model pick just Age f1 score'


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


seed_everything(seed)

In [ ]:

dict_args = {
'seed' : seed,
'train_b_size': train_b_size,
'epochs' : epochs,
'lr' : lr,
'model_name' : model_name,
'num_labels' : num_labels,
'train_ratio': train_ratio,
'loss_function_name' : loss_function_name,
'optimizer_name' : optimizer_name,
'weight_decay' : weight_decay,
'scheduler_name' : scheduler_name,
'comment' : comment
}

In [ ]:
os.makedirs(save_dir,exist_ok=True)

In [ ]:
with open(os.path.join(save_dir, 'config.json'), 'w', encoding='utf-8') as f:
        json.dump(dict_args, f, ensure_ascii=False, indent=4)

## datasets class

In [ ]:
class CustomTrainDataset(Dataset):
    ## input pd.Series
    ## output np.ndarray
    ## change dummy, if label_col is 'gender' or 'mask_state'
    def __init__(self, img_paths : pd.Series, labels : pd.Series, label_col='class', transforms=None):
        self.img_paths = img_paths.to_numpy()
        self.transforms = transforms
        if label_col == 'gender':
            self.labels = pd.get_dummies(labels).to_numpy()
        elif label_col == 'mask_state':
            self.labels = pd.get_dummies(labels).to_numpy()
        else: # age, classes
            self.labels = labels.to_numpy(dtype=np.float32)
        ## if (False), assert occur
        assert self.transforms is not None, 'you must use transforms in Trainset'
    
    ## return numpy img, label
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = np.array(Image.open(img_path))

        img = self.transforms(image=img)["image"]
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.labels)

## augmentations

In [ ]:
train_transforms = A.Compose([
        A.CenterCrop(height=450,width=250),
        A.HorizontalFlip(p=0.5),
        A.Resize(224, 224),
        A.CLAHE(p=.5,clip_limit=6.0),
        # A.ToGray(p=.3),
        # A.ColorJitter(p=.3),
        A.GridDistortion(p=.5),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2, p=1),
            A.RandomContrast(limit=.2, p=1),
            A.RandomGamma(p=1)
        ], p=.3),
        A.OneOf([
            A.Blur(blur_limit=3, p=1),
            A.MedianBlur(blur_limit=3, p=1)
        ], p=.2),
        A.OneOf([
            A.GaussNoise(0.002, p=.5),
        ], p=.2),
        A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
        ToTensorV2()
    ])
    
valid_transforms = A.Compose([
        A.CenterCrop(height=450,width=250),
        A.Resize(224, 224),
        A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
        ToTensorV2()
    ])

## make dataset and dataloader

In [ ]:
df_train = pd.read_csv('./dataset/train/hypothesis_train.csv')
df_valid = pd.read_csv('./dataset/train/hypothesis_valid.csv')

In [ ]:
## num of dset
print(len(df_train), len(df_valid))

In [ ]:
df_train.head()

In [ ]:
def age_to_class(age):
    cls = None
    if age<30:
        cls = 0
    elif 30<=age<60:
        cls = 1
    elif age>=60:
        cls = 2
    else:
        cls = None
    if cls == None:
        assert ValueError
    return cls


In [ ]:
age_train_cls = []
age_valid_cls = []

for age in df_train['age']:
    age_train_cls.append(age_to_class(age))

for age in df_valid['age']:
    age_valid_cls.append(age_to_class(age))

In [ ]:
df_train['age_cls'] = age_train_cls
df_valid['age_cls'] = age_valid_cls

In [ ]:
df_train

In [ ]:
df_valid

In [ ]:
df_train[['mask_class','gender_class','age_cls','age']]
df_valid[['mask_class','gender_class','age_cls','age']]

In [ ]:
train_dset = CustomTrainDataset(df_train['path'], df_train[['mask_class','gender_class','age_cls','age']], 'class', train_transforms)
val_dset = CustomTrainDataset(df_valid['path'], df_valid[['mask_class','gender_class','age_cls','age']], 'class', valid_transforms)

In [ ]:
print(train_dset[0][1])

In [ ]:
## num of iter(batches)
print(len(train_dset), len(val_dset))

In [ ]:
train_loader = DataLoader(
        train_dset,
        batch_size=train_b_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=True,
        pin_memory=use_cuda,
        drop_last=False,
    )

val_loader = DataLoader(
        val_dset,
        batch_size=valid_b_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

In [ ]:
print(len(train_loader))
print(len(val_loader))

## visualization transform

In [ ]:
## for check transform
imgs, labels = next(iter(train_loader))
plt.figure(figsize=(10, 5))
plt.imshow(make_grid(imgs, normalize=True).permute(1,2,0))
plt.show()

In [ ]:
## for check transform
imgs, labels = next(iter(train_loader))
plt.figure(figsize=(10, 5))
plt.imshow(make_grid(imgs, normalize=True).permute(1,2,0))
plt.show()

In [ ]:
# imgs, labels = next(iter(val_loader))
# plt.figure(figsize=(10, 5))
# plt.imshow(make_grid(imgs, normalize=True).permute(1,2,0))
# plt.show()

## Model

In [ ]:
timm.list_models('*convnext*', pretrained=True)

In [ ]:
# model = timm.create_model('swin_base_patch4_window7_224_in22k', pretrained=True, num_classes=num_labels)
# model.to(device)

In [ ]:
# print(summary(model,torch.Tensor(2,3,224,224).to(device)))

In [ ]:
model = timm.create_model(model_name=model_name, pretrained=True, num_classes=num_labels)
model.to(device)

In [ ]:
print(summary(model,torch.Tensor(2,3,224,224).to(device)))

## Loss function

In [ ]:
num_of_cls = df_train['age_cls'].value_counts().sort_index().values.tolist()
age_loss_weight = torch.tensor([max(num_of_cls)/i for i in num_of_cls]).to(device)
num_of_cls = df_train['mask_class'].value_counts().sort_index().values.tolist()
mask_loss_weight = torch.tensor([max(num_of_cls)/i for i in num_of_cls]).to(device)
num_of_cls = df_train['gender_class'].value_counts().sort_index().values.tolist()
gender_loss_weight = torch.tensor([max(num_of_cls)/i for i in num_of_cls]).to(device)

In [ ]:
df_train['age_cls'].value_counts().sort_index().values.tolist()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

class F1Loss(nn.Module):
    def __init__(self, classes=18, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

class L1LossFlat(nn.SmoothL1Loss):
    def forward(self, input:torch.Tensor, target:torch.Tensor):
        return super().forward(input.view(-1), target.view(-1))

In [ ]:
## for mask and gender    [1.,5.,5.]).to(device)
age_criterion = nn.CrossEntropyLoss(weight=age_loss_weight)
mask_criterion = nn.CrossEntropyLoss(weight=mask_loss_weight) # classes=3
gender_criterion = nn.CrossEntropyLoss(weight=gender_loss_weight)
age_reg_criterion = L1LossFlat()

## Optimizer

In [ ]:
# if you param freeze, not update during training
optimizer = None
if optimizer_name == 'AdamW':
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay) # 0.09 
else:
    raise ValueError(f'not implement Optimizer : {optimizer_name}')

In [ ]:
optimizer

## Scheduler  

In [ ]:
scheduler = None

if scheduler_name == 'ReduceLROnPlateau':
    ## during 5epochs, valid loss decrease 1e-3↓, lr update lr*0.5
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold=1e-3) ## 정리
elif scheduler_name == 'CosineAnnealingWarmRestarts':
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-4)

assert scheduler != None , 'sheduler is None'

In [ ]:
scheduler

## Train

In [ ]:
def age_to_class(age, std=0):
    cls = None
    if age<30:
        cls = 0
    elif 30<=age<60-std:
        cls = 1
    elif age>=60-std:
        cls = 2
    else:
        cls = None
    if cls == None:
        assert ValueError
    return cls

In [ ]:
task_weight = [1., 1., 1., 0.]
# task_weight = [i/min(task_weight) for i in task_weight]
# norm = [(float(i)-min(task_weight))/(max(task_weight)-min(task_weight)) for i in task_weight]

In [ ]:
for w in task_weight:
    print(w)

In [ ]:
## for True Label
def get_label(mask_state, gender, age_cls):
    label = 0
    if mask_state == 0 and gender == 0 and age_cls == 0: # 0:
        label = 0
    elif mask_state == 0 and gender == 0 and age_cls == 1: # 1
        label = 1
    elif mask_state == 0 and gender == 0 and age_cls == 2: # 2
        label = 2
    elif mask_state == 0 and gender == 1 and age_cls == 0: # 3
        label = 3
    elif mask_state == 0 and gender == 1 and age_cls == 1: # 4
        label = 4
    elif mask_state == 0 and gender == 1 and age_cls == 2: # 5
        label = 5
    elif mask_state == 1 and gender == 0 and age_cls == 0: # 6
        label = 6
    elif mask_state == 1 and gender == 0 and age_cls == 1: # 7
        label = 7
    elif mask_state == 1 and gender == 0 and age_cls == 2: # 8
        label = 8
    elif mask_state == 1 and gender == 1 and age_cls == 0: # 9
        label = 9
    elif mask_state == 1 and gender == 1 and age_cls == 1: # 10
        label = 10
    elif mask_state == 1 and gender == 1 and age_cls == 2: # 11
        label = 11
    elif mask_state == 2 and gender == 0 and age_cls == 0: # 12
        label = 12
    elif mask_state == 2 and gender == 0 and age_cls == 1: # 13
        label = 13
    elif mask_state == 2 and gender == 0 and age_cls == 2: # 14
        label = 14
    elif mask_state == 2 and gender == 1 and age_cls == 0: # 15
        label = 15
    elif mask_state == 2 and gender == 1 and age_cls == 1: # 16
        label = 16
    elif mask_state == 2 and gender == 1 and age_cls == 2: # 17
        label = 17
    else:
        raise ValueError
    return label

In [ ]:
def get_label_another(mask_state, gender, age_reg):
    label = 0
    if mask_state == 0 and gender == 0 and age_reg<30: # 0:
        label = 0
    elif mask_state == 0 and gender == 0 and 30<=age_reg<60: # 1
        label = 1
    elif mask_state == 0 and gender == 0 and age_reg>=60: # 2
        label = 2
    elif mask_state == 0 and gender == 1 and age_reg<30: # 3
        label = 3
    elif mask_state == 0 and gender == 1 and 30<=age_reg<60: # 4
        label = 4
    elif mask_state == 0 and gender == 1 and age_reg>=60: # 5
        label = 5
    elif mask_state == 1 and gender == 0 and age_reg<30: # 6
        label = 6
    elif mask_state == 1 and gender == 0 and 30<=age_reg<60: # 7
        label = 7
    elif mask_state == 1 and gender == 0 and age_reg>=60: # 8
        label = 8
    elif mask_state == 1 and gender == 1 and age_reg<30: # 9
        label = 9
    elif mask_state == 1 and gender == 1 and 30<=age_reg<60: # 10
        label = 10
    elif mask_state == 1 and gender == 1 and age_reg>=60: # 11
        label = 11
    elif mask_state == 2 and gender == 0 and age_reg<30: # 12
        label = 12
    elif mask_state == 2 and gender == 0 and 30<=age_reg<60: # 13
        label = 13
    elif mask_state == 2 and gender == 0 and age_reg>=60: # 14
        label = 14
    elif mask_state == 2 and gender == 1 and age_reg<30: # 15
        label = 15
    elif mask_state == 2 and gender == 1 and 30<=age_reg<60: # 16
        label = 16
    elif mask_state == 2 and gender == 1 and age_reg>=60: # 17
        label = 17
    else:
        raise ValueError
    return label

In [ ]:
best_val_loss = np.Inf
best_val_acc = 0.
best_val_f1 = 0.

start=time.process_time()

for epoch in range(epochs):
    model.train()

    epoch_loss = 0
    epoch_mask_loss = 0
    epoch_gender_loss = 0
    epoch_age_loss = 0
    epoch_age_reg_loss = 0
    

    epoch_mask_cls_preds = []
    epoch_mask_cls_labels = []

    epoch_gender_cls_preds = []
    epoch_gender_cls_labels = []

    epoch_age_cls_preds = []
    epoch_age_cls_labels = []

    epoch_pred_ages = []
    epoch_label_ages = []


    for idx, train_batch in enumerate(train_loader):
        b_imgs, f_labels = train_batch # batch imgs and batch labels
        b_imgs = b_imgs.to(device)

        b_mask_labels, b_gender_labels, b_age_labels, b_age_reg_labels = torch.split(f_labels,[1,1,1,1], dim=1)

        b_mask_labels = b_mask_labels.squeeze().to(device).to(torch.int64)
        b_gender_labels = b_gender_labels.squeeze().to(device).to(torch.int64)
        b_age_labels = b_age_labels.squeeze().to(device).to(torch.int64)
        b_age_reg_labels = b_age_reg_labels.to(device).to(torch.float32)

        b_output = model(b_imgs)
        
        b_mask_logit, b_gender_logit, b_age_logit, b_age_reg_pred = torch.split(b_output, [3,2,3,1], dim=1)

        mask_loss = mask_criterion(b_mask_logit.to(device), b_mask_labels.squeeze())
        gender_loss = gender_criterion(b_gender_logit.to(device), b_gender_labels.squeeze())
        age_loss = age_criterion(b_age_logit.to(device), b_age_labels.squeeze())
        age_reg_loss = age_reg_criterion(b_age_reg_pred.to(device), b_age_reg_labels)
    
        b_loss = mask_loss*task_weight[0] + \
                    gender_loss*task_weight[1] + \
                    age_loss*task_weight[2] + \
                    age_reg_loss*task_weight[3]

        optimizer.zero_grad()
        b_loss.backward()
        optimizer.step()
        
        epoch_loss += b_loss.item()

        epoch_mask_loss += mask_loss.item()*task_weight[0]
        epoch_gender_loss += gender_loss.item()*task_weight[1]
        epoch_age_loss += age_loss.item()*task_weight[2]
        epoch_age_reg_loss += age_reg_loss.item()*task_weight[3]

        b_mask_preds = torch.argmax(b_mask_logit, dim=-1)
        b_gender_preds = torch.argmax(b_gender_logit, dim=-1)
        b_age_preds = torch.argmax(b_age_logit, dim=-1)
        

        epoch_mask_cls_preds += b_mask_preds.detach().cpu().numpy().flatten().tolist()
        epoch_mask_cls_labels += b_mask_labels.detach().cpu().numpy().flatten().tolist()

        epoch_gender_cls_preds += b_gender_preds.detach().cpu().numpy().flatten().tolist()
        epoch_gender_cls_labels += b_gender_labels.detach().cpu().numpy().flatten().tolist()

        epoch_age_cls_preds += b_age_preds.detach().cpu().numpy().flatten().tolist()
        epoch_age_cls_labels += b_age_labels.detach().cpu().numpy().flatten().tolist() 

        epoch_pred_ages += b_age_reg_pred.detach().cpu().numpy().flatten().tolist()
        epoch_label_ages += b_age_reg_labels.detach().cpu().numpy().flatten().tolist() # for 1d dim

        # print interval batch
        if(idx+1) % print_interval == 0:

            current_loss = epoch_loss / (idx+1) # / batch
            current_mask_loss = epoch_mask_loss/(idx+1)
            current_gender_loss = epoch_gender_loss/(idx+1)
            current_age_loss = epoch_age_loss/(idx+1)
            currnet_age_reg_loss = epoch_age_reg_loss/(idx+1)

            pred_cls = [get_label(m,g,a) for (m,g,a) in zip(epoch_mask_cls_preds, epoch_gender_cls_preds, epoch_age_cls_preds)]
            label_cls = [get_label(m,g,a) for (m,g,a) in zip(epoch_mask_cls_labels, epoch_gender_cls_labels, epoch_age_cls_labels)]
            
            true_f1 = f1_score(label_cls, pred_cls, average='macro')
            age_f1 = f1_score(epoch_mask_cls_labels, epoch_age_cls_preds, average='macro')
            epoch_MAE = mean_absolute_error(epoch_label_ages, epoch_pred_ages)
            print(f"Epoch[{epoch+1}/{epochs}]({idx + 1}/{len(train_loader)}) || "
                  f"training loss {current_loss:2.4f} "
                  f"M {current_mask_loss:2.4f} G {current_gender_loss:2.4f} A {current_age_loss:2.4f} AR {currnet_age_reg_loss:2.4f} ||")
            print(f"True f1 {true_f1:2.4f} , age f1 {age_f1:2.4f} || train MAE {epoch_MAE:2.4f} || "
                  f"lr {optimizer.param_groups[0]['lr']:.5f}")
                
    
    scheduler.step()
    
    with torch.no_grad():
        print("    Calculating validation results...")
        model.eval()

        epoch_v_age_loss = 0
        epoch_v_age_reg_loss = 0

        val_loss = []
        val_age_reg_loss = []


        val_mask_cls_preds = []
        val_mask_cls_labels = []

        val_gender_cls_preds = []
        val_gender_cls_labels = []

        val_age_cls_preds = [] 
        val_age_cls_labels = []

        val_preds_reg_age = []
        val_labels_reg_age = []

        for idx, val_batch in enumerate(val_loader):
            imgs, f_labels = val_batch
            mask_labels, gender_labels, age_labels, age_reg_labels = torch.split(f_labels,[1,1,1,1], dim=1)

            imgs = imgs.to(device)
            
            mask_labels = mask_labels.squeeze().to(device).to(torch.int64)
            gender_labels = gender_labels.squeeze().to(device).to(torch.int64)
            age_labels = age_labels.squeeze().to(device).to(torch.int64)
            age_reg_labels = age_reg_labels.to(device).to(torch.float32)

            outputs = model(imgs)

            mask_logit, gender_logit, age_logit, age_reg_p = torch.split(outputs,[3,2,3,1], dim=1)

            mask_loss = mask_criterion(mask_logit.to(device), mask_labels.squeeze())
            gender_loss = gender_criterion(gender_logit.to(device), gender_labels.squeeze())
            age_loss = age_criterion(age_logit, age_labels.squeeze())
            age_reg_loss = age_reg_criterion(age_reg_p.to(device), age_reg_labels)
            
            t_loss = mask_loss*task_weight[0] + \
                     gender_loss*task_weight[1] + \
                     age_loss*task_weight[2] + \
                     age_reg_loss*task_weight[3]
            
            val_loss.append(t_loss.item())

            epoch_v_age_loss += age_loss.item()*task_weight[2]
            epoch_v_age_reg_loss += age_reg_loss.item()*task_weight[3]

            mask_preds = torch.argmax(mask_logit, dim=-1)
            gender_preds = torch.argmax(gender_logit, dim=-1)
            age_preds = torch.argmax(age_logit, dim=-1)

            val_mask_cls_preds += mask_preds.detach().cpu().numpy().flatten().tolist()
            val_mask_cls_labels += mask_labels.detach().cpu().numpy().flatten().tolist()

            val_gender_cls_preds += gender_preds.detach().cpu().numpy().flatten().tolist()
            val_gender_cls_labels += gender_labels.detach().cpu().numpy().flatten().tolist()

            val_age_cls_preds += age_preds.detach().cpu().numpy().flatten().tolist()
            val_age_cls_labels += age_labels.detach().cpu().numpy().flatten().tolist()

            val_preds_reg_age += age_reg_p.detach().cpu().numpy().flatten().tolist()
            val_labels_reg_age += age_reg_labels.detach().cpu().numpy().flatten().tolist()

        pred_cls = [get_label(m,g,a) for (m,g,a) in zip(val_mask_cls_preds, val_gender_cls_preds, val_age_cls_preds)]
        label_cls = [get_label(m,g,a) for (m,g,a) in zip(val_mask_cls_labels, val_gender_cls_labels, val_age_cls_labels)]

        epoch_val_f1 = f1_score(label_cls, pred_cls, average='macro')
        epoch_val_f1_age = f1_score(val_age_cls_labels, val_age_cls_preds, average='macro')

        epoch_val_loss = sum(val_loss)/len(val_loss)
        current_age_loss = epoch_v_age_loss / len(val_loss)
        current_age_reg_loss = epoch_v_age_reg_loss/len(val_loss)

        best_val_loss = min(best_val_loss, epoch_val_loss)

        if best_val_f1 < epoch_val_f1:
                print(f"    ★ New best model for True val f1 : {epoch_val_f1:2.4f}! saving the best model..")
                best_val_f1 = epoch_val_f1
                torch.save(model.state_dict(), f"{save_dir}/best.pth")
                
                list_total_preds = pred_cls # if best score val-set, assign pred result to list_total_preds
                list_total_labels = label_cls # if best score val-set, assign pred result to list_total_preds

                df_diff = pd.DataFrame()
                df_diff['labels'] = list_total_labels
                df_diff['preds'] = list_total_preds
                df_diff.to_csv(f'{save_dir}/diff.csv')
                
        torch.save(model.state_dict(), f'{save_dir}/last.pth')
        print(f"    [{epoch+1} epoch Val] True f1:{epoch_val_f1:2.4f}, Age f1:{epoch_val_f1_age:2.4f}, loss, age_loss, age_reg_loss:{epoch_val_loss:2.2f}, {current_age_loss:2.2f}, {current_age_reg_loss:2.2f} || ")
        print(f"    best loss:{best_val_loss:2.4f}, Best f1:{best_val_f1:2.4f}")
    print()
    src = './9_T4064_Experiment_4hd.ipynb'
    dst = f"{save_dir}/code.ipynb"
    shutil.copy(src,dst)
end = time.process_time()

## Result

In [ ]:
print("Time elapsed: ", timedelta(seconds=end-start))

## Error Analysis

In [ ]:
df_diff = pd.DataFrame()
df_diff['labels'] = list_total_labels
df_diff['preds'] = list_total_preds
df_diff.to_csv(f'{save_dir}/diff.csv')

In [ ]:
df_diff[df_diff['labels']!=df_diff['preds']].value_counts().sort_index()

In [ ]:
sum(df_diff[df_diff['labels']!=df_diff['preds']].value_counts().values)

In [ ]:
100-255/len(df_diff)*100

In [ ]:
src = './9_T4064_Experiment_4hd.ipynb'
dst = f"{save_dir}/code.ipynb"
shutil.copy(src,dst)